# Avocado Sales in US cities

Data set sources:

- Avocado Sales: https://www.kaggle.com/datasets/neuromusic/avocado-prices?resource=download

- US Cities: https://simplemaps.com/data/us-cities

## Data Preparation and Cleaning

In [1]:
# import packages

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
# 1. download dataset from kaggle https://www.kaggle.com/datasets/neuromusic/avocado-prices?resource=download

# 2. import donwnloaded dataset 
df_avocado = pd.read_csv(r"...\avocado.csv")
df_avocado.head()


,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
# print amount of unique regions
print(f"# unique regions: {df_avocado['region'].nunique()}") # 54 unique regions

# print list of unique regions
regions_unique = df_avocado['region'].unique()
print(regions_unique)

# unique regions: 54
['Albany' 'Atlanta' 'BaltimoreWashington' 'Boise' 'Boston'
 'BuffaloRochester' 'California' 'Charlotte' 'Chicago' 'CincinnatiDayton'
 'Columbus' 'DallasFtWorth' 'Denver' 'Detroit' 'GrandRapids' 'GreatLakes'
 'HarrisburgScranton' 'HartfordSpringfield' 'Houston' 'Indianapolis'
 'Jacksonville' 'LasVegas' 'LosAngeles' 'Louisville' 'MiamiFtLauderdale'
 'Midsouth' 'Nashville' 'NewOrleansMobile' 'NewYork' 'Northeast'
 'NorthernNewEngland' 'Orlando' 'Philadelphia' 'PhoenixTucson'
 'Pittsburgh' 'Plains' 'Portland' 'RaleighGreensboro' 'RichmondNorfolk'
 'Roanoke' 'Sacramento' 'SanDiego' 'SanFrancisco' 'Seattle'
 'SouthCarolina' 'SouthCentral' 'Southeast' 'Spokane' 'StLouis' 'Syracuse'
 'Tampa' 'TotalUS' 'West' 'WestTexNewMexico']


In [5]:
# clean up region names
df_avocado_clean = df_avocado.copy()

# function to split combined region names
def split_region(region):
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', region)

# apply the split_region function to the 'region' column
df_avocado_clean['region'] = df_avocado['region'].apply(split_region)
print(df_avocado_clean['region'].unique())

# assign values to only one city at a time (e.g. Baltimore Washington --> Baltimore)
city_mapping = {
    'Baltimore Washington': 'Baltimore',
    'Buffalo Rochester': 'Buffalo',
    'Cincinnati Dayton': 'Cincinnati',
    'Dallas Ft Worth': 'Dallas',
    'Harrisburg Scranton': 'Harrisburg',
    'Hartford Springfield': 'Hartford',
    'Miami Ft Lauderdale': 'Miami',
    'New Orleans Mobile': 'New Orleans',
    'Phoenix Tucson': 'Phoenix',
    'Raleigh Greensboro': 'Raleigh',
    'Richmond Norfolk': 'Richmond',
    'St Louis': 'St. Louis'
}

df_avocado_clean['region'] = df_avocado_clean['region'].replace(city_mapping)
print(df_avocado_clean['region'].unique()) # check if mapping was successful


['Albany' 'Atlanta' 'Baltimore Washington' 'Boise' 'Boston'
 'Buffalo Rochester' 'California' 'Charlotte' 'Chicago'
 'Cincinnati Dayton' 'Columbus' 'Dallas Ft Worth' 'Denver' 'Detroit'
 'Grand Rapids' 'Great Lakes' 'Harrisburg Scranton' 'Hartford Springfield'
 'Houston' 'Indianapolis' 'Jacksonville' 'Las Vegas' 'Los Angeles'
 'Louisville' 'Miami Ft Lauderdale' 'Midsouth' 'Nashville'
 'New Orleans Mobile' 'New York' 'Northeast' 'Northern New England'
 'Orlando' 'Philadelphia' 'Phoenix Tucson' 'Pittsburgh' 'Plains'
 'Portland' 'Raleigh Greensboro' 'Richmond Norfolk' 'Roanoke' 'Sacramento'
 'San Diego' 'San Francisco' 'Seattle' 'South Carolina' 'South Central'
 'Southeast' 'Spokane' 'St Louis' 'Syracuse' 'Tampa' 'Total US' 'West'
 'West Tex New Mexico']


array(['Albany', 'Atlanta', 'Baltimore', 'Boise', 'Boston', 'Buffalo',
       'California', 'Charlotte', 'Chicago', 'Cincinnati', 'Columbus',
       'Dallas', 'Denver', 'Detroit', 'Grand Rapids', 'Great Lakes',
       'Harrisburg', 'Hartford', 'Houston', 'Indianapolis',
       'Jacksonville', 'Las Vegas', 'Los Angeles', 'Louisville', 'Miami',
       'Midsouth', 'Nashville', 'New Orleans', 'New York', 'Northeast',
       'Northern New England', 'Orlando', 'Philadelphia', 'Phoenix',
       'Pittsburgh', 'Plains', 'Portland', 'Raleigh', 'Richmond',
       'Roanoke', 'Sacramento', 'San Diego', 'San Francisco', 'Seattle',
       'South Carolina', 'South Central', 'Southeast', 'Spokane',
       'St. Louis', 'Syracuse', 'Tampa', 'Total US', 'West',
       'West Tex New Mexico'], dtype=object)

In [5]:
# I only want to keep data for cities, not for states etc.: save rows that are not cities in a separate dataframe
not_cities = ['California', 'Great Lakes', 'Midsouth', 'Northeast', 'Northern New England', 'South Carolina', 'South Central', 'Southeast', 'Total US', 'West', 'West Tex New Mexico']
df_avocado_not_cities = df_avocado_clean[df_avocado_clean['region'].isin(not_cities)]
df_avocado_clean = df_avocado_clean[~df_avocado_clean['region'].isin(not_cities)]

df_avocado_not_cities.to_csv(r"...\avocado_not_city.csv", index=False)

In [6]:
# 1. download dataset from simplemaps https://simplemaps.com/data/us-cities

# 2. import downloaded dataset
df_cities = pd.read_csv(r"...\uscities.csv")

print(f"# cities: {len(df_cities)}") # several cities have the same name, but are in different states
df_cities.head()

31120


,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11922389,3184.7,shape,False,True,America/Los_Angeles,1,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8497759,4614.5,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6080145,4758.9,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5970127,1384.0,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925


In [7]:
# since the avocado data was conducted in large cities/regions, I only keep the first row of each city name which corresponds to the largest city with this name (correct matching of the cities was randomly checked)
df_cities = df_cities.drop_duplicates(subset='city', keep='first') # only keep first row of each city name
print(f"# cities after dropping: {len(df_cities)}") 

# cities after dropping: 20855


In [9]:
# merge df_cities to df_avocado_clean on common column city & region
df_merged = pd.merge(df_avocado_clean, df_cities, left_on='region', right_on='city', how='left')
df_merged = df_merged.drop(columns=['Unnamed: 0'])
df_merged.to_csv(r"...\avocado_cities.csv", index=False)
df_merged.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,...,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,...,-73.7987,590694.0,1798.3,shape,False,True,America/New_York,2.0,12208 12209 12204 12206 12207 12202 12203 1220...,1.840000e+09
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,...,-73.7987,590694.0,1798.3,shape,False,True,America/New_York,2.0,12208 12209 12204 12206 12207 12202 12203 1220...,1.840000e+09
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,...,-73.7987,590694.0,1798.3,shape,False,True,America/New_York,2.0,12208 12209 12204 12206 12207 12202 12203 1220...,1.840000e+09
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,...,-73.7987,590694.0,1798.3,shape,False,True,America/New_York,2.0,12208 12209 12204 12206 12207 12202 12203 1220...,1.840000e+09
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,...,-73.7987,590694.0,1798.3,shape,False,True,America/New_York,2.0,12208 12209 12204 12206 12207 12202 12203 1220...,1.840000e+09


In [10]:
# change structure of dataframe for pie chart in Tableau --> change to long format (new column for weight type)
df_long = pd.melt(df_avocado_clean, 
                  id_vars=['Unnamed: 0', 'Date', 'AveragePrice', 'Total Volume', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year', 'region'],
                  value_vars=['4046', '4225', '4770'],
                  var_name='weight_type',
                  value_name='sold_volume_weight')

# drop the 'Unnamed: 0' column
df_long = df_long.drop(columns=['Unnamed: 0'])
df_long.head()

# save to csv
df_long.to_csv(r"...\avocado_weight_pie.csv", index=False)

,Date,AveragePrice,Total Volume,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,weight_type,sold_volume_weight
0,2015-12-27,1.33,64236.62,8696.87,8603.62,93.25,0.0,conventional,2015,Albany,4046,1036.74
1,2015-12-20,1.35,54876.98,9505.56,9408.07,97.49,0.0,conventional,2015,Albany,4046,674.28
2,2015-12-13,0.93,118220.22,8145.35,8042.21,103.14,0.0,conventional,2015,Albany,4046,794.70
3,2015-12-06,1.08,78992.15,5811.16,5677.40,133.76,0.0,conventional,2015,Albany,4046,1132.00
4,2015-11-29,1.28,51039.60,6183.95,5986.26,197.69,0.0,conventional,2015,Albany,4046,941.48
